In [1]:
import os
os.chdir("../..")

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent, Fleet
from shell.fleet.grad.gradient_fleet import GradFleet
from shell.fleet.data.data_utilize import *
from shell.fleet.data.send import *
from shell.fleet.grad.monograd import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
seed_everything(0)

In [4]:
use_contrastive = True
num_tasks = 5
num_init_tasks = 3
dataset_name = "mnist"
num_epochs = 20
com_frequency = 5

In [5]:
data_cfg = {
    "dataset_name": dataset_name,
    "num_tasks": num_tasks,
    "num_train_per_task": 128,
    "num_val_per_task": 102,
    'remap_labels': True,
    'use_contrastive': use_contrastive,
}

joint_dataset = get_dataset(**data_cfg)

dataset = get_dataset(**data_cfg)
dataset2 = get_dataset(**data_cfg)
dataset3 = get_dataset(**data_cfg)

INFO:root:Class sequence: [5 0 3 4 2 6 7 1 8 9]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)
INFO:root:Class sequence: [1 9 8 4 5 2 6 3 0 7]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)
INFO:root:Class sequence: [6 0 1 3 7 2 4 9 8 5]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)
INFO:root:Class sequence: [6 8 0 9 5 7 1 3 2 4]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28)


In [6]:
net_cfg = {
    'depth': num_init_tasks,
    'layer_size': 64,
    'num_init_tasks': num_init_tasks,
    'i_size': 28,
    'num_classes': 2,
    'num_tasks': 4,
    'dropout': 0.0,
}

agent_cfg = {
    'memory_size': 64,
    'use_contrastive': use_contrastive,
    'save_dir': 'test',
    'dataset_name': dataset_name,
}

In [7]:
NetCls = MLP
LearnerCls = NoComponentsER

# NetCls = MLPSoftLLDynamic
# LearnerCls = CompositionalDynamicER

AgentCls = ModelSyncAgent
sharing_cfg = DictConfig({
    'num_coms_per_round': 1,
    "comm_freq": com_frequency,
    "log_freq": 1,

})
train_cfg = {
    "num_epochs": num_epochs,
}

In [8]:
datasets = [dataset, dataset2, dataset3]
g = TopologyGenerator(num_nodes=len(datasets)).generate_fully_connected()
fleet = GradFleet(g, 0, datasets,
              sharing_cfg, AgentCls, NetCls, LearnerCls, net_cfg, agent_cfg, 
              train_cfg,
              joint_dataset)

INFO:root:Agent: node_id: 69420, seed: 69420000
INFO:root:Agent: node_id: 0, seed: 0
INFO:root:Agent: node_id: 1, seed: 1000
INFO:root:Agent: node_id: 2, seed: 2000
INFO:root:Created fleet with 3 agents
INFO:root:Adding neighbors...
INFO:root:Fleet initialized
INFO:root:Joint training on task 0 ...
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.69359534	acc: 0.476
INFO:root:	task: avg	loss: 0.69359534	acc: 0.476
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.69359534	acc: 0.476
INFO:root:	task: avg	loss: 0.69359534	acc: 0.476
INFO:root:Joint training on task 1 ...
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.69359534	acc: 0.476
INFO:root:	task: 1	loss: 0.69507260	acc: 0.493
INFO:root:	task: avg	loss: 0.69433397	acc: 0.485
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.69359534	acc: 0.476
INFO:root:	task: 1	loss: 0.69507260	acc: 0.493
INFO:root:	task: avg	loss: 0.69433397	acc: 0.485
INFO:root:Joint training on tas

In [9]:
fleet.train_and_comm(0)

INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.70071523	acc: 0.529
INFO:root:	task: avg	loss: 0.70071523	acc: 0.529
INFO:root:epochs: 1, training task: 0
INFO:root:	task: 0	loss: 0.70325607	acc: 0.529
INFO:root:	task: avg	loss: 0.70325607	acc: 0.529
INFO:root:epochs: 2, training task: 0
INFO:root:	task: 0	loss: 0.70337497	acc: 0.529
INFO:root:	task: avg	loss: 0.70337497	acc: 0.529
INFO:root:epochs: 3, training task: 0
INFO:root:	task: 0	loss: 0.70146407	acc: 0.529
INFO:root:	task: avg	loss: 0.70146407	acc: 0.529
INFO:root:epochs: 4, training task: 0
INFO:root:	task: 0	loss: 0.69837832	acc: 0.529
INFO:root:	task: avg	loss: 0.69837832	acc: 0.529
INFO:root:epochs: 5, training task: 0
INFO:root:	task: 0	loss: 0.69572289	acc: 0.529
INFO:root:	task: avg	loss: 0.69572289	acc: 0.529
INFO:root:epochs: 6, training task: 0
INFO:root:	task: 0	loss: 0.69572289	acc: 0.529
INFO:root:	task: avg	loss: 0.69572289	acc: 0.529
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss:

In [10]:
fleet.agents[0].net.components[0].bias

Parameter containing:
tensor([ 0.0003,  0.0640,  0.0597, -0.0257, -0.0820, -0.0110,  0.0963, -0.1202,
         0.0987, -0.0132, -0.0302,  0.0641, -0.0059,  0.1188,  0.0209,  0.0280,
        -0.0810,  0.0588,  0.1494, -0.0248,  0.0596,  0.1331, -0.0671,  0.1003,
        -0.1333,  0.1131, -0.0971, -0.0930, -0.0376, -0.0975, -0.0704, -0.0724,
         0.0962, -0.0033,  0.0660, -0.0511, -0.0304, -0.0528,  0.0185, -0.0748,
        -0.0477,  0.0628, -0.0443, -0.0252,  0.1229,  0.0219,  0.0543,  0.1200,
        -0.0210,  0.0804,  0.0012,  0.0301, -0.0963, -0.0650,  0.0219,  0.0961,
         0.0347,  0.0958, -0.0597,  0.0975,  0.0868,  0.1034,  0.0697,  0.1085],
       device='cuda:0', requires_grad=True)

In [11]:
fleet.agents[2].net.components[0].bias

Parameter containing:
tensor([ 0.0003,  0.0640,  0.0597, -0.0257, -0.0820, -0.0110,  0.0963, -0.1202,
         0.0987, -0.0132, -0.0302,  0.0641, -0.0059,  0.1188,  0.0209,  0.0280,
        -0.0810,  0.0588,  0.1494, -0.0248,  0.0596,  0.1331, -0.0671,  0.1003,
        -0.1333,  0.1131, -0.0971, -0.0930, -0.0376, -0.0975, -0.0704, -0.0724,
         0.0962, -0.0033,  0.0660, -0.0511, -0.0304, -0.0528,  0.0185, -0.0748,
        -0.0477,  0.0628, -0.0443, -0.0252,  0.1229,  0.0219,  0.0543,  0.1200,
        -0.0210,  0.0804,  0.0012,  0.0301, -0.0963, -0.0650,  0.0219,  0.0961,
         0.0347,  0.0958, -0.0597,  0.0975,  0.0868,  0.1034,  0.0697,  0.1085],
       device='cuda:0', requires_grad=True)

In [12]:
# original_num_epochs = 100
# communication_frequency = 100


# def communicate():
#     print('commute')
# for start_epoch in range(0, original_num_epochs, communication_frequency):
#     end_epoch = min(start_epoch + communication_frequency, original_num_epochs)
#     adjusted_num_epochs = end_epoch - start_epoch  # adjust num_epochs
#     print(start_epoch, end_epoch, adjusted_num_epochs)
#     communicate()